# Pitfall: Forget the Colon When Declaring a Function

A common pitfall is forgetting to end the function declaration with colon. Note that the "invalid syntax" message usually does not tell new-comer anything useful:

In [1]:
def greet(name)
    print('Hello', name)
    
greet('world')

SyntaxError: invalid syntax (<ipython-input-1-a62f28931079>, line 1)

# Tip: Function that does not Return a Value

If a function does not return a value, the interpreter will act as if the function returns a `None` value:

In [3]:
def add(a, b):
    c = a + b
    
print(add(3, 5))
print(add(3, 5) is None)

None
True


Notes

* There is no way to distinguish between a `None` returned explicitly or implicitly
* If a function meants to return None, do it explicitely

# Tip: Communicate "Not Found" Condition

Many search functions return `None` to indicate not-found condition. While this approach works well in general, you should be aware of some corner cases. For example, the following function looks up a dictionary for a specific key and return its value (we pretend that the `dict.get` method does not exist:

In [2]:
def dict_get(dictobj, target_key):
    for key, value in dictobj.items():
        if target_key == key:
            return value
    return None

passwords = dict(john='i4Got', karen='uCanNotG355')
print('Password for Alex: {}'.format(dict_get(passwords, 'alex')))

Password for Alex: None


This function will not work if `None` is a value in the dictionary:

In [3]:
passwords = dict(john='i4Got', karen='uCanNotG355', alex=None)
print(passwords)
print('Password for Alex: {}'.format(dict_get(passwords, 'alex')))

{'karen': 'uCanNotG355', 'alex': None, 'john': 'i4Got'}
Password for Alex: None


In the previous example, we cannot tell if the password for Alex is `None`, or Alex is not in the dictionary. One way to deal with this situation is provide a default value in case that a key is not found--this is how the `dict.get` method works:

In [9]:
print('Without default: {}'.format(passwords.get('paul')))
print('With default: {}'.format(passwords.get('paul', 'Paul is not found')))

Without default: None
With default: Paul is not found


One last problem: in the last line, what if Paul is in the dictionary and his password is actually *'Paul is not found'*? How do we deal with this? The answer is to create a unique value (a sentinel) as default. Note that we are using the `is` operator for indentity test (as opposed to value test):

In [10]:
NOT_FOUND = object()
pauls_password = passwords.get('paul', NOT_FOUND)
if pauls_password is NOT_FOUND:
    print('Paul is not in the system')
else:
    print("Paul's password is {!r}".format(pauls_password))    

Paul is not in the system


# Optional, Default Parameter

Consider a function to convert a string to integer:

In [21]:
def str2int(str_value):
    return int(str_value)

print(str2int('-19'))

-19


What if we want to be able to specify a base (2, 8, 10, 16, ...)? Where the most common one is base 10. One way to achieve this goal is to use default parameter:

In [22]:
def str2int(str_value, base=10):
    return int(str_value, base)

print(str2int('11'))      # Base 10, the default
print(str2int('11', 16))  # Base 16

11
17


Notes

* Introducing the optional, default parameter is a good way to retrofit a function without making changes everywhere
* The default parameter must be of the value that we are using the most

# Pitfall: Mutable Default Parameter

Consider the following function:

In [36]:
def list_append(item, listobj=[]):
    listobj.append(item)
    return listobj

print(list_append(5))
print(list_append(9))

[5]
[5, 9]


To avoid this problem, we should use `None` as the default value:

In [37]:
def list_append(item, listobj=None):
    if listobj is None:
        listobj = []
        
    listobj.append(item)
    return listobj

print(list_append(5))
print(list_append(9))

[5]
[9]


Note that many experience use this effect in order to "remember" previous values. Consider a print function which only print each value once:

In [39]:
def print_once(value, previous_values=set()):
    if value in previous_values:
        return
    previous_values.add(value)  # Cache it
    print('Value: {}, cache: {}'.format(value, previous_values))
    
print_once(1)
print_once(2)
print_once(3)
print_once(1)
print_once(2)
print_once(3)

Value: 1, cache: {1}
Value: 2, cache: {1, 2}
Value: 3, cache: {1, 2, 3}


# Tip: Use Named Argument for Clarity

I often see code that looks like this:

    sheet = workbook.goto_sheet('Sheet 1', True)
    
Consider the second parameter, does `True` tell us anything? Unless you are familiar with this method, you don't know what `True` means in this case. How about this:

    sheet = workbook.goto_sheet('Sheet 1', return_sheet=True)
    
With named argument, we can tell right away the second argument's role. I recommend to use named argument to make code easier to understand.


# Tip: Format Function Calls with Many Arguments

Consider the following function call:

    selection = SelectionPresModel.from_attributes(i_ds=annotation_ids, node_selections=[], oriented_node_selections=[], type=SelectionType.ANNOTATIONS)
    
In order to keep the code readable, we should break the line down to several shorter lines. There are two recommended ways to format your code. The first is to line up the arguments right after the left parenthesis:

    selection = SelectionPresModel.from_attributes(i_ds=annotation_ids,
                                                   node_selections=[],
                                                   oriented_node_selections=[],
                                                   type=SelectionType.ANNOTATIONS)

The second is to indent all arguments:

    selection = SelectionPresModel.from_attributes(
        i_ds=annotation_ids,
        node_selections=[],
        oriented_node_selections=[],
        type=SelectionType.ANNOTATIONS)

The first approach has the advantage to keep things clean, while the second works better with arguments that are long. Personally, I prefer a variation of the second approach:

    selection = SelectionPresModel.from_attributes(
        i_ds=annotation_ids,
        node_selections=[],
        oriented_node_selections=[],
        type=SelectionType.ANNOTATIONS,
    )

Note that the last named argument is followed by a comma, just like the lines above it. This is perfectly alright in Python and in fact encourage. The reason for this last comma is to have consistency among the arguments. Plus, adding and removing parameter is easier.

# Callable in Python

In other languages, a "callable" mostly means functions and methods. In Python, the concept of callable is broader, which includes:

* Functions and methods
* Classes (by means of `__new__` and `__init__` magic methods
* Object instances (when a class implements the `__call__` magic method

To test to see if an object is callable, we can use the builtin function `callable` which returns a boolean.

In [7]:
class Animal(object):
    pass

class Tag(object):
    def __init__(self, tag_name):
        self.tag_name = tag_name
        
    def __call__(self, body):
        print('<{}>'.format(self.tag_name))
        print(body)
        print('</{}>'.format(self.tag_name))

def greet(name):
    print('Hello', name)
    
name = 'John'

tag = Tag('p')
tag('Lorem ipsum dolor sit amet, consectetur adipiscing elit.')
print('---')

print('greet (function) is callable:', callable(greet))
print('name (string) is callable:', callable(name))
print('Animal (class) is callable:', callable(Animal))
print('tag (object) is callable:', callable(tag))


<p>
Lorem ipsum dolor sit amet, consectetur adipiscing elit.
</p>
---
greet (function) is callable: True
name (string) is callable: False
Animal (class) is callable: True
tag (object) is callable: True


# Recipe: Create Function that Takes any Number of Parameters